In [2]:
import pandas as pd
from pandas.io.json import json_normalize
pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
import shutil
import os
from pathlib import Path
import requests
import json
import csv
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#import plotly.express as px
#import plotly.graph_objs as go
#import matplotlib.pyplot as plt

import streamlit as st
import time

pd.options.mode.chained_assignment = None

In [9]:
strava_activities_raw = pd.read_csv('strava_activities.csv')

In [10]:
cols_to_drop = ['Unnamed: 0', 'resource_state', 'start_date_local', 'timezone', 
'utc_offset', 'trainer', 'commute', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
'average_watts', 'kilojoules', 'device_watts', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
'has_heartrate', 'location_city', 'location_state', 'location_country']

strava_activities = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

In [11]:
strava_activities.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,achievement_count,...,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,has_kudoed,suffer_score
0,Afternoon Run,9684.7,3682,3685,314.0,Run,Run,7395188303,2022-06-30T21:39:30Z,11,...,4.392,84.4,28.0,154.1,180.0,183.2,48.2,6,False,102.0
1,Afternoon Golf,10366.5,10586,14280,120.0,Golf,Golf,7390944431,2022-06-30T00:12:17Z,0,...,2.624,NaN,28.0,102.5,148.0,93.8,68.4,0,False,29.0
2,Lunch Run,6529.2,2165,2168,132.0,Run,Run,7389543804,2022-06-29T19:24:38Z,3,...,4.302,87.0,27.0,150.8,167.0,181.4,49.0,0,False,47.0
3,Lunch Run,9723.3,3264,3304,61.0,Run,Run,7384051727,2022-06-28T19:05:39Z,0,...,4.058,85.2,23.0,149.2,159.0,83.6,45.6,0,False,73.0
4,Morning Run,32196.4,12853,12931,362.0,Run,Run,7373215314,2022-06-26T13:58:16Z,26,...,4.490,85.8,25.0,145.7,163.0,258.0,87.6,11,False,229.0


In [4]:
def get_strava_data():
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.DataFrame()

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')
            
        
        
        if (not r):   # if no results then exit loop
            break
        
        if not bool([v for v in list(strava_activities_raw_1.id) if v not in list(test_str_activities_raw.id)]):
            break

        activities = pd.concat([activities, r_df])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)    
    return activities

In [124]:
# TESTING WHAT THE RESPONSE IS, AKA WHAT THAT r COMES BACK LOOKING LIKE

with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)


if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']

activities = pd.DataFrame()
r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
r = r.json()

# the old method: 
#activities = pd.concat([activities, json_normalize(r)])
#activities.to_csv('strava_activities_raw_1.csv')

# the new method: 
activities = pd.concat([activities, pd.DataFrame.from_dict(r, orient='columns')])
activities.to_csv('strava_activities_raw_2.csv', index=False)


In [156]:
#set(strava_activities_raw_1.id) - set(test_str_activities_raw.id)

bool([v for v in list(strava_activities_raw_1.id) if v not in list(test_str_activities_raw.id)])

True

In [145]:
if []:
    print('nah')

In [107]:
#strava_activities_raw_1 = pd.read_csv('strava_activities_raw_1.csv')
strava_activities_raw_2 = pd.read_csv('strava_activities_raw_2.csv')

In [122]:
# always remember to pass index=False to not add an Unnamed column on your export! 


In [114]:
strava_activities_raw_1.head(3)['average_speed']

0    2.630
1    0.979
2    3.016
Name: average_speed, dtype: float64

In [125]:
test_str_activities_raw = strava_activities_raw_2.iloc[3:]

In [126]:
activity_ids = list(test_str_activities_raw['id'])

In [127]:
strava_activities_raw_2.shape

(200, 56)

In [128]:
test_str_activities_raw.shape

(197, 56)

In [131]:
strava_activities_raw_2.head(5)

,Unnamed: 0,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
0,0,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Run,9684.7,3682,3685,314.0,Run,Run,...,7880648682,garmin_push_9117457633,False,6,0,False,102.0,NaN,NaN,NaN
1,1,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Golf,10366.5,10586,14280,120.0,Golf,Golf,...,7876007090,garmin_push_9112316946,False,0,0,False,29.0,NaN,NaN,NaN
2,2,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,6529.2,2165,2168,132.0,Run,Run,...,7874481805,garmin_push_9110840088,False,0,0,False,47.0,NaN,NaN,NaN
3,3,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,9723.3,3264,3304,61.0,Run,Run,...,7868484844,garmin_push_9104450794,False,0,0,False,73.0,NaN,NaN,NaN
4,4,2,"{'id': 23871091, 'resource_state': 1}",Morning Run,32196.4,12853,12931,362.0,Run,Run,...,7856623564,garmin_push_9091593487,False,11,0,False,229.0,NaN,NaN,NaN


In [130]:
test_str_activities_raw.head(3)

,Unnamed: 0,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,...,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,average_watts,kilojoules,device_watts
3,3,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,9723.3,3264,3304,61.0,Run,Run,...,7868484844,garmin_push_9104450794,False,0,0,False,73.0,NaN,NaN,NaN
4,4,2,"{'id': 23871091, 'resource_state': 1}",Morning Run,32196.4,12853,12931,362.0,Run,Run,...,7856623564,garmin_push_9091593487,False,11,0,False,229.0,NaN,NaN,NaN
5,5,2,"{'id': 23871091, 'resource_state': 1}",Lunch Run,9789.8,3128,3162,64.0,Run,Run,...,7844706162,garmin_push_9079167003,False,0,0,False,63.0,NaN,NaN,NaN


In [87]:
# I want to time these two different things: iterate over the df adding only the new rows
# or just appending the response then dropping dupes immediately

new_test_raw = pd.concat([response_df, test_str_activities_raw])

In [96]:
new_test_raw.shape

(800, 61)

In [95]:
new_test_raw.drop_duplicates(subset=['id'], inplace=True)

In [79]:

for activity in response_json:           #[0]['id']
    print(activity['id'])
    if activity['id'] in activity_ids:
        break
    else:
        pd.concat([json_normalize(activity), test_str_activities_raw], axis=0)


7395188303
7390944431
7389543804
7384051727


C:\Users\dansu\AppData\Local\Temp\ipykernel_12012\206313412.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  pd.concat([json_normalize(activity), test_str_activities_raw], axis=0)
C:\Users\dansu\AppData\Local\Temp\ipykernel_12012\206313412.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  pd.concat([json_normalize(activity), test_str_activities_raw], axis=0)
C:\Users\dansu\AppData\Local\Temp\ipykernel_12012\206313412.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  pd.concat([json_normalize(activity), test_str_activities_raw], axis=0)


In [52]:
test_str_activities_raw['id']

3      7384051727
4      7373215314
5      7362386558
6      7357786241
7      7352038723
8      7348290559
9      7342540098
10     7340486405
11     7319569765
12     7314918293
13     7309774433
14     7303783051
15     7299973019
16     7293707995
17     7288063515
18     7282237232
19     7273314296
20     7266468382
21     7256215176
22     7245056529
23     7240188526
24     7233936674
25     7229942915
26     7224096501
27     7218835356
28     7212574267
29     7207093585
30     7202278601
31     7197345247
32     7193030113
33     7192680297
34     7186813405
35     7181506998
36     7181017312
37     7180581474
38     7176084890
39     7171636545
40     7165512705
41     7159904786
42     7154042525
43     7137152715
44     7132913288
45     7127836348
46     7124993827
47     7122669524
48     7115364421
49     7095254686
50     7088839177
51     7083425271
52     7077910274
53     7067108450
54     7056622874
55     7051376234
56     7044719126
57     7040848623
58     703

In [7]:
#strava_activities = get_strava_data().to_csv('strava_activities.csv')

strava_activities = pd.read_csv('strava_activities.csv')

In [6]:
strava_activities.sport_type.unique()

array(['Run', 'Ride', 'Walk', 'Golf', 'Hike', 'NordicSki', 'AlpineSki',
       'Elliptical', 'Rowing', 'StandUpPaddling', 'Kayaking'],
      dtype=object)

In [8]:
strava_activities.head()

,Unnamed: 0,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_watts,kilojoules,device_watts
0,0,2,Afternoon Run,9684.7,3682,3685,314.0,Run,Run,NaN,...,False,102.0,23871091,1,a7395188303,u{ztGxwhkVTE|@E`AApAJlA?p@Fh@@l@@h@ExA@pBJdA@p...,2,NaN,NaN,NaN
1,1,2,Afternoon Golf,10366.5,10586,14280,120.0,Golf,Golf,NaN,...,False,29.0,23871091,1,a7390944431,ea{tGzhxjVALGCHAEFICNGEa@aAkGDYNDKDUaADI_@Y_Aq...,2,NaN,NaN,NaN
2,2,2,Lunch Run,6529.2,2165,2168,132.0,Run,Run,NaN,...,False,47.0,23871091,1,a7389543804,szztGfvhkVRDR?NEbBF|BAV@^HX@j@Gf@Bh@CnADzAC\@D...,2,NaN,NaN,NaN
3,3,2,Lunch Run,9723.3,3264,3304,61.0,Run,Run,NaN,...,False,73.0,23871091,1,a7384051727,}_{tGbxhkVIQcAEe@Fu@AyAJq@AWC_@KOA{@EqB@sCDcEC...,2,NaN,NaN,NaN
4,4,2,Morning Run,32196.4,12853,12931,362.0,Run,Run,NaN,...,False,229.0,23871091,1,a7373215314,cm}tGjq`lV~ErGt@lBuHl@{CkAqRlBs@b@SvAkCM_JvHWl...,2,NaN,NaN,NaN
